In [3]:
"""
generate_2025_data.py
---------------------
Generates synthetic 2025 retail data based on the original 2024 dataset.

Methodology:
- Uses pathlib for OS-agnostic path handling.
- Applies category-level revenue growth factors.
- Applies monthly volume multipliers to break the mirror effect between years.
- Adds small noise to unit-level fields for realism.
- Outputs a combined 2024+2025 CSV for Power BI ingestion.
"""

import pandas as pd
import numpy as np
from pathlib import Path

# ── Config ────────────────────────────────────────────────────────────────────

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

BASE_DIR = Path(__file__).resolve().parent
INPUT_FILE  = BASE_DIR / "Business_Analytics_Dataset_10000_Rows.csv"
OUTPUT_FILE = BASE_DIR / "Business_Analytics_Dataset_Final.csv"

# Category-level YoY revenue growth factors
GROWTH_FACTORS = {
    "Beauty":         1.04,
    "Clothing":        1.02,
    "Electronics":     0.97,
    "Home & Kitchen": 1.03,
    "Sports":          0.98,
}

# Monthly volume multipliers controls order count per month in 2025
MONTHLY_VOLUME = {
    1:  0.95, 2:  1.18, 3:  1.09, 4:  1.04, 
    5:  0.86, 6:  0.86, 7:  0.82, 8:  1.15, 
    9:  1.04, 10: 1.08, 11: 1.05, 12: 1.03,
}

if not INPUT_FILE.exists():
    print(f"Error: {INPUT_FILE.name} not found in {BASE_DIR}")
    exit()

df_all = pd.read_csv(INPUT_FILE)
df_all.columns = df_all.columns.str.strip().str.replace(";", "")
df_all["Order_Date"] = pd.to_datetime(df_all["Order_Date"])

# Extract only 2024 as the base
df_2024 = df_all[df_all["Order_Date"].dt.year == 2024].copy().reset_index(drop=True)
print(f"Base dataset loaded: {len(df_2024)} rows (2024)")


def safe_replace_year(d):
    """Handle Feb 29 edge case when shifting year."""
    try:
        return d.replace(year=2025)
    except ValueError:
        return d.replace(year=2025, day=28)

frames = []

for month, volume_factor in MONTHLY_VOLUME.items():
    base = df_2024[df_2024["Order_Date"].dt.month == month].copy().reset_index(drop=True)
    n = max(1, int(round(len(base) * volume_factor)))

    # Sample with replacement if growing, without if shrinking
    sampled = base.sample(n=n, replace=(n > len(base)), random_state=month).reset_index(drop=True)
    sampled["Order_Date"] = sampled["Order_Date"].apply(safe_replace_year)
    frames.append(sampled)

df_2025 = pd.concat(frames, ignore_index=True)
df_2025 = df_2025.sample(frac=1, random_state=99).reset_index(drop=True)

df_2025["Order_ID"] = range(df_2024["Order_ID"].max() + 1, df_2024["Order_ID"].max() + 1 + len(df_2025))
df_2025["Customer_ID"] = [f"CUST{np.random.randint(1000, 9999)}" for _ in range(len(df_2025))]
df_2025["Payment_Method"] = df_2025["Payment_Method"].sample(frac=1, random_state=7).reset_index(drop=True)

n = len(df_2025)
factors = np.array([
    GROWTH_FACTORS[df_2025.at[i, "Product_Category"]] * np.random.uniform(0.98, 1.02)
    for i in range(n)
])

df_2025["Revenue"]       = (df_2025["Revenue"].values * factors).round(2)
df_2025["Cost"]          = (df_2025["Cost"].values * factors * np.random.uniform(0.99, 1.01, size=n)).round(2)
df_2025["Profit"]        = (df_2025["Revenue"] - df_2025["Cost"]).round(2)
df_2025["Unit_Price"]    = (df_2025["Unit_Price"].values * np.random.uniform(0.99, 1.01, size=n)).round(2)
df_2025["Quantity"]      = np.maximum(1, (df_2025["Quantity"].values * np.random.uniform(0.97, 1.03, size=n)).round().astype(int))
df_2025["Discount_Rate"] = np.clip(df_2025["Discount_Rate"].values + np.random.uniform(-0.01, 0.01, size=n), 0.01, 0.45).round(2)

combined = pd.concat([df_2024, df_2025], ignore_index=True)
combined["Order_Date"] = combined["Order_Date"].dt.strftime("%Y-%m-%d")
combined.to_csv(OUTPUT_FILE, index=False)

print(f"\nSuccess! Processed {len(combined)} total rows.")
print(f"Final file exported to: {OUTPUT_FILE}")

NameError: name '__file__' is not defined